# Import statements

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import os

import tensorflow as tf
from IPython.display import display, Image

import pathlib
import glob
import pickle
from imutils import paths
import cv2
#import tensorflow_datasets as tfds
from keras.models import Model, Sequential
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Dense, Dropout,merge,Reshape, Activation
from keras.layers.normalization import BatchNormalization
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.callbacks import TensorBoard
import random

# Load the data

In [0]:
!wget http://data.csail.mit.edu/places/places365/val_large.tar

In [0]:
import tarfile
tar = tarfile.open("val_large.tar")
tar.extractall()

In [0]:
!wget http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip
!wget http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_HR.zip

!mkdir div2k
!unzip -q DIV2K_train_HR.zip -d div2k
!unzip -q DIV2K_valid_HR.zip -d div2k
!ls

## train data

### data from the div2k dataset

In [0]:
 imgPaths = list(paths.list_images(r"./div2k/DIV2K_train_HR/"))
 print(imgPaths)
 print(len(imgPaths))
 photosPaths = list(glob.glob('./div2k/DIV2K_train_HR/*.png'))
 print(len(photosPaths))

### data from the Places dataset

In [0]:
 photosPaths1 = list(glob.glob('./val_large/*.jpg'))
 print(len(photosPaths1))

## valid data

In [0]:
 imgPaths2 = list(paths.list_images(r"./div2k/DIV2K_valid_HR/"))
 print(imgPaths2)
 print(len(imgPaths2))
 photosPaths2 = list(glob.glob('./div2k/DIV2K_valid_HR/*.png'))
 print(len(photosPaths2))

## display 2 images


In [0]:
#display 2 images
sample_images = glob.glob('./div2k/DIV2K_train_HR/*.png')[:2]
for file_path in sample_images:
  display(Image(file_path))

# Process the data

## Photos without noise

### Data from the div2k dataset

In [0]:
data_train = []
for imgPath in photosPaths: 
   img = cv2.imread(imgPath, cv2.IMREAD_COLOR)
   img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
   img = cv2.resize(img,(200,200))
   img = np.asarray(img)
   img = img.astype(np.float32)
   img= img/255.0
   data_train.append(img)
data_train=np.array(data_train)
#print(data[0])
print(len(data_train))
print(data_train.shape)

data_valid = []
for imgPath in photosPaths2: 
   img = cv2.imread(imgPath, cv2.IMREAD_COLOR)
   img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
   img = cv2.resize(img,(200,200))
   img = np.asarray(img)
   img = img.astype(np.float32)
   img= img/255.0
   data_valid.append(img)
data_valid=np.array(data_valid)

### Data from the places dataset

In [0]:
data_train2 = []
# not enough ram to use all the images...
for i in range (3000): 
   img = cv2.imread(photosPaths1[i], cv2.IMREAD_COLOR)
   img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
   img = cv2.resize(img,(200,200))
   img = np.asarray(img)
   img = img.astype(np.float32)
   img= img/255.0
   data_train2.append(img)
data_train2=np.array(data_train2)

data_valid2 = []
for i in range (3000): 
   img = cv2.imread(photosPaths1[i+3001], cv2.IMREAD_COLOR)
   img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
   img = cv2.resize(img,(200,200))
   img = np.asarray(img)
   img = img.astype(np.float32)
   img= img/255.0
   data_valid2.append(img)
data_valid2=np.array(data_valid2)

## Photos with noise

### Data from the div2k dataset

In [0]:
noise_factor = 0.2
data_train_noise = data_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=data_train.shape)
data_valid_noise = data_valid + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=data_valid.shape)
data_train_noise = np.clip(data_train_noise, 0., 1.)
data_valid_noise = np.clip(data_valid_noise, 0., 1.)


In [0]:
print("max : ",np.max(data_train))
print("min : ",np.min(data_train))
print("max : ",np.max(data_valid))
print("min : ",np.min(data_valid))

### Data from the places dataset

In [0]:
noise_factor = 0.2
data_train2_noise = data_train2 + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=data_train2.shape)
data_valid2_noise = data_valid2 + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=data_valid2.shape)
data_train2_noise = np.clip(data_train2_noise, 0., 1.)
data_valid2_noise = np.clip(data_valid2_noise, 0., 1.)

# Autoencoder

##Create the model

In [0]:
batch_size = 200
epochs = 20
input_img = Input(shape = (200, 200, 3))

def autoencoder(input_img):
    #encoder
    c1 = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    pool1 = MaxPooling2D(pool_size=(2, 2))(c1)
    c2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2))(c2)
    encoded = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)

    #decoder
    c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(encoded)
    up1 = UpSampling2D((2,2))(c4)
    c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(up1)
    up2 = UpSampling2D((2,2))(c5)
    decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(up2)
    return decoded

In [0]:
autoencoder = Model(input_img, autoencoder(input_img))


## Compile the model

In [0]:
autoencoder.compile(loss='mean_squared_error', optimizer = tf.keras.optimizers.RMSprop())
autoencoder.summary()

## Train the model

### Data from the div2k dataset

In [0]:
autoencoder_train = autoencoder.fit(data_train_noise, data_train, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(data_valid_noise,data_valid))

### Data from the places dataset

In [0]:
autoencoder_train = autoencoder.fit(data_train2_noise, data_train2, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(data_valid2_noise,data_valid2))

# Test the model

## Plot the training and validation loss

In [0]:
loss = autoencoder_train.history['loss']
val_loss = autoencoder_train.history['val_loss']
epochs = range(epochs)
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

## Show some result images

### decoding the validation images just to see how it looked

In [0]:
decoded_imgs = autoencoder.predict(data_valid_noise)

In [0]:
n = 5
plt.figure(figsize=(15,8))
for i in range(n):
    #disp original
    ax = plt.subplot(2, n, i+1)
    plt.imshow(data_valid_noise[i].reshape(200,200,3))
    plt.plasma()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i+n+1)
    plt.imshow(decoded_imgs[i].reshape(200,200,3))
    plt.plasma()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

### Using images from the places dataset to test the model trained wth div2k images

In [0]:
data_test = []
for i in range (20): 
   img = cv2.imread(photosPaths1[i], cv2.IMREAD_COLOR)
   img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
   img = cv2.resize(img,(200,200))
   img = np.asarray(img)
   img = img.astype(np.float32)
   img= img/255.0
   data_test.append(img)
data_test=np.array(data_test)

In [0]:
noise_factor = 0.2
data_test_noise = data_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=data_test.shape)
data_test_noise = np.clip(data_test_noise, 0., 1.)

In [0]:
decoded_imgs = autoencoder.predict(data_test_noise)

In [0]:
n = 5
plt.figure(figsize=(15,8))
for i in range(n):
    #disp original
    ax = plt.subplot(2, n, i+1)
    plt.imshow(data_test[i].reshape(200,200,3))
    plt.plasma()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i+n+1)
    plt.imshow(decoded_imgs[i].reshape(200,200,3))
    plt.plasma()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()